# 神经机器翻译系统

用TensorFlow搭建seq2seq模型实现了一个简单的神经机器翻译系统，实现英语翻译为法语。Encoder使用双向LSTM。Decoder采用了attention机制。

使用多张计算图分别处理train，eval和infer，并分别在不同的session中进行训练和推断。参数共享用Saver。

decoder的initial state采用双向encoder state的平均值。

详见TensorFlow教程：https://tensorflow.google.cn/tutorials/seq2seq

### 导入包
检查TensorFlow版本和GPU情况

In [1]:
from distutils.version import LooseVersion
import warnings, os
import numpy as np
import tensorflow as tf
from tqdm import tqdm

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow Version: 1.7.0


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: No GPU found. Please use a GPU to train your neural network.
  if sys.path[0] == '':


### 超参数设置

In [2]:
source_path = 'en-fr/small_vocab_en'
target_path = 'en-fr/small_vocab_fr'
checkpoint_path = './tmp-model.ckpt'
batch_size = 256
num_units = 64
max_gradient_norm = 5.0
learning_rate = 0.001
epoch = 3

### 建立lookup table文件
sos句子开始。eos句子结束。

In [3]:
l=[]
with open(source_path, 'r', encoding='utf-8') as f:
    for line in f.readlines():
        l += line.split()
f.close()
unique_words_src = ['eos'] + list(set(l))

with open('en-fr/words_en', 'w', encoding='utf-8') as f:
    for word in unique_words_src:
        f.write(word + '\n')
f.close()

In [4]:
l=[]
with open(target_path, 'r', encoding='utf-8') as f:
    for line in f.readlines():
        l += line.split()
f.close()
unique_words_tar = ['sos'] + ['eos'] + list(set(l))
print(len(l), len(unique_words_tar))
#print ('rusty' in unique_words_en)

with open('en-fr/words_fr', 'w', encoding='utf-8') as f:
    for word in unique_words_tar:
        f.write(word + '\n')
f.close()

1961295 357


### 使用预训练词向量 FastText

In [5]:
embed_file_src = os.path.join('.', 'fasttext', 'wiki-news-300d-1M.vec')
embed_file_tar = os.path.join('.', 'fasttext', 'cc.fr.300.vec')
embed_size = 300

In [6]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index_src = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(embed_file_src, encoding = 'utf-8'))
embeddings_index_tar = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(embed_file_tar, encoding = 'utf-8'))

#### 建立两种语言的embedding matrix

In [7]:
embedding_matrix_src = np.random.normal(size=(len(unique_words_src), embed_size), scale=0.01)
embedding_matrix_tar = np.random.normal(size=(len(unique_words_tar), embed_size), scale=0.01)

for i, word in enumerate(unique_words_src):
    embedding_vector = embeddings_index_src.get(word)
    if embedding_vector is not None: embedding_matrix_src[i] = embedding_vector
        
for i, word in enumerate(unique_words_tar):
    embedding_vector = embeddings_index_tar.get(word)
    if embedding_vector is not None: embedding_matrix_tar[i] = embedding_vector

### 生成lookup table函数

In [8]:
def BuildLookupTable(source_words_path, target_words_path):
    lookup_src = tf.contrib.lookup.index_table_from_file(source_words_path)
    lookup_tar = tf.contrib.lookup.index_table_from_file(target_words_path)
    lookup_translate = tf.contrib.lookup.index_to_string_table_from_file(target_words_path)
    return lookup_src, lookup_tar, lookup_translate

### 输入训练文本预处理函数
预处理source和target dataset。文本转成单词id。target开头加一个sos。分batch并pad。末尾用eos补足到最大长度。
这里不需要drop remainder，iterator会自动计算最后一批的样本量。但是后面不能再使用batch size

In [9]:
def BuildTrainDataset(source_path, target_path, lookup_src, lookup_tar, src_eos_id, tar_eos_id):
    
    # source
    source_dataset = tf.data.TextLineDataset(source_path)
    source_dataset = source_dataset.map(lambda string: tf.string_split([string]).values)
    source_dataset = source_dataset.map(lambda words: (words, tf.size(words)))
    source_dataset = source_dataset.map(lambda words, size: (lookup_src.lookup(words), size))
    
    # target
    target_dataset = tf.data.TextLineDataset(target_path)
    target_dataset = target_dataset.map(lambda string: tf.string_split([tf.string_join([tf.constant('sos'), string], separator=' ')]).values)
    target_dataset = target_dataset.map(lambda words: (words, tf.size(words)))
    target_dataset = target_dataset.map(lambda words, size: (lookup_tar.lookup(words), size))
    
    # zip source and target
    source_target_dataset = tf.data.Dataset.zip((source_dataset, target_dataset))

    # batch and pad
    batched_dataset = source_target_dataset.padded_batch(
        batch_size,
        padded_shapes=((tf.TensorShape([None]),  # source vectors of unknown size
                        tf.TensorShape([])),     # size(source)
                       (tf.TensorShape([None]),  # target vectors of unknown size
                        tf.TensorShape([]))),    # size(target)
        padding_values=((src_eos_id,  # source vectors padded on the right with src_eos_id
                         0),          # size(source) -- unused
                        (tar_eos_id,  # target vectors padded on the right with tar_eos_id
                         0)))         # size(target) -- unused
    
    return batched_dataset

### Build the train model function
Input: batched and padded dataset iterator ((source, source_lengths), (target, target_lengths))

Output: command to run in train session

In [10]:
def BuildTrainModel(train_iterator):
    ((source, source_lengths), (target, target_lengths)) = train_iterator.get_next()
    encoder_inputs = tf.transpose(source, [1,0])
    decoder_inputs = tf.transpose(target, [1,0])
    decoder_outputs = tf.pad(decoder_inputs[1:], tf.constant([[0,1],[0,0]]), constant_values=tar_eos_id)

    shape = tf.shape(decoder_outputs)
    target_weights = tf.to_double(tf.where(tf.equal(decoder_outputs, tf.fill(shape, tar_eos_id)), tf.zeros(shape), tf.ones(shape)))
            
    embedding_encoder = tf.Variable(embedding_matrix_src, name='embedding_encoder')
    embedding_decoder = tf.Variable(embedding_matrix_tar, name='embedding_decoder')
    
    # Embedding layer
    encoder_emb_inp = tf.nn.embedding_lookup(embedding_encoder, encoder_inputs)
    decoder_emb_inp = tf.nn.embedding_lookup(embedding_decoder, decoder_inputs)
    
    # Encoder
    # Construct forward and backward cells
    forward_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units, name = 'forward_cell')
    backward_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units, name = 'backward_cell')

    bi_outputs, encoder_state = tf.nn.bidirectional_dynamic_rnn(
        forward_cell, backward_cell, encoder_emb_inp, dtype=tf.float64,
        sequence_length=source_lengths, time_major=True)
    encoder_outputs = tf.concat(bi_outputs, -1)
    #encoder_state: tuple of 2 LSTM state tuple ((cell_state_fw, hidden_state_fw), (cell_state_bw, hidden_state_bw))
    
    # Attention
    attention_states = tf.transpose(encoder_outputs, [1, 0, 2])
    attention_mechanism = tf.contrib.seq2seq.LuongAttention(
        num_units, attention_states, memory_sequence_length=source_lengths, dtype=tf.float64)
    decoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units, name = 'decoder_cell')
    decoder_cell = tf.contrib.seq2seq.AttentionWrapper(decoder_cell, attention_mechanism, attention_layer_size=num_units)
    initial_state = decoder_cell.zero_state(dtype=tf.float64, batch_size=tf.shape(encoder_inputs)[1])
    # Use the average of the forward and backward states as the decoder initial state
    initial_state = initial_state.clone(
        cell_state = tf.contrib.rnn.LSTMStateTuple((encoder_state[0].c+encoder_state[1].c)/2.0, (encoder_state[0].h+encoder_state[1].h)/2.0))
    
    # Projection layer on the top
    projection_layer = tf.layers.Dense(len(unique_words_tar), use_bias=False, name='projection')
    
    # Decoder for training
    helper = tf.contrib.seq2seq.TrainingHelper(decoder_emb_inp, target_lengths, time_major=True)
    decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, helper, initial_state, output_layer=projection_layer)
    outputs, final_state, final_sequence_lengths = tf.contrib.seq2seq.dynamic_decode(decoder, output_time_major=True, impute_finished=True)
    logits = outputs.rnn_output
    
    # Loss
    crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=decoder_outputs, logits=logits)
    train_loss = tf.reduce_sum(crossent * target_weights)/ tf.to_double(tf.shape(encoder_inputs)[1])
    
    # Gradient
    params = tf.trainable_variables()
    gradients = tf.gradients(train_loss, params)
    clipped_gradients, _ = tf.clip_by_global_norm(gradients, max_gradient_norm)
    
    # Optimization
    optimizer = tf.train.AdamOptimizer(learning_rate)
    update_step = optimizer.apply_gradients(zip(clipped_gradients, params))
    
    return train_loss, update_step

### 设置train graph

In [14]:
train_graph = tf.Graph()
with train_graph.as_default():
    
    # Build the lookup table
    lookup_src, lookup_tar, lookup_translate = BuildLookupTable('en-fr/words_en', 'en-fr/words_fr')
    
    # set the sos and eos
    src_eos_id=lookup_src.lookup(tf.constant('eos')) #0 in source vocab
    tar_sos_id=lookup_tar.lookup(tf.constant('sos')) #0 in target vocab
    tar_eos_id=lookup_tar.lookup(tf.constant('eos')) #1 in target vocab
    
    # Preprocess the text dataset
    batched_dataset = BuildTrainDataset(source_path, target_path, lookup_src, lookup_tar, src_eos_id, tar_eos_id)
    
    # Build the train model
    train_iterator = batched_dataset.make_initializable_iterator()
    train_model = BuildTrainModel(train_iterator)
    initializer = tf.global_variables_initializer()
    table_initializer = tf.tables_initializer()
    train_saver = tf.train.Saver(max_to_keep=2)

### Run the train session to train the model and save the variables

In [15]:
train_sess = tf.Session(graph=train_graph)
train_sess.run(initializer)
train_sess.run(table_initializer)
train_sess.run(train_iterator.initializer)

#saver.restore(sess, './tmp-model.ckpt-11')
for i in tqdm(range(epoch)):
    train_sess.run(train_iterator.initializer)
    n_batch=0
    while True:
        try:
            cost, _ = train_sess.run(train_model)
            n_batch+=1
            print (n_batch)
            if n_batch % 10 == 0:
                print (cost)
        except tf.errors.OutOfRangeError:
            print (cost)
            break
    model_path = train_saver.save(train_sess, checkpoint_path, global_step=i+1)



  0%|                                                    | 0/3 [00:00<?, ?it/s]

1
2
3
4
5
6
7
8
9
10
74.09925021489255
11
12
13
14
15
16
17
18
19
20
61.91181622826241
21
22
23
24
25
26
27
28
29
30
56.83938282776754
31
32
33
34
35
36
37
38
39
40
53.25028692388574
41
42
43
44
45
46
47
48
49
50
47.93364744736162
51
52
53
54
55
56
57
58
59
60
46.46141099592222
61
62
63
64
65
66
67
68
69
70
41.41327376713903
71
72
73
74
75
76
77
78
79
80
38.81239173957978
81
82
83
84
85
86
87
88
89
90
34.88848848055166
91
92
93
94
95
96
97
98
99
100
31.21750597459757
101
102
103
104
105
106
107
108
109
110
28.676892182143852
111
112
113
114
115
116
117
118
119
120
25.73639324913146
121
122
123
124
125
126
127
128
129
130
23.06338152249214
131
132
133
134
135
136
137
138
139
140
21.284231311758262
141
142
143
144
145
146
147
148
149
150
19.643886822323097
151
152
153
154
155
156
157
158
159
160
17.620755053275953
161
162
163
164
165
166
167
168
169
170
16.125990177434524
171
172
173
174
175
176
177
178
179
180
14.25927913900631
181
182
183
184
185
186
187
188
189
190
12.824387098460361




 33%|██████████████▎                            | 1/3 [13:48<27:36, 828.49s/it]

1
2
3
4
5
6
7
8
9
10
1.5233878123776097
11
12
13
14
15
16
17
18
19
20
1.5627409848245686
21
22
23
24
25
26
27
28
29
30
1.4389145382240058
31
32
33
34
35
36
37
38
39
40
1.171213870314539
41
42
43
44
45
46
47
48
49
50
1.3867937729575095
51
52
53
54
55
56
57
58
59
60
1.2238911384564612
61
62
63
64
65
66
67
68
69
70
1.1403359185755029
71
72
73
74
75
76
77
78
79
80
1.4659976488894098
81
82
83
84
85
86
87
88
89
90
1.2293411236949239
91
92
93
94
95
96
97
98
99
100
1.491033089718093
101
102
103
104
105
106
107
108
109
110
1.2450086192931233
111
112
113
114
115
116
117
118
119
120
1.0972165569343346
121
122
123
124
125
126
127
128
129
130
1.220610457981576
131
132
133
134
135
136
137
138
139
140
1.2460613694751337
141
142
143
144
145
146
147
148
149
150
1.3535079647487098
151
152
153
154
155
156
157
158
159
160
1.3425752696619335
161
162
163
164
165
166
167
168
169
170
1.347572492800169
171
172
173
174
175
176
177
178
179
180
1.1658870853779333
181
182
183
184
185
186
187
188
189
190
0.98740635



 67%|████████████████████████████▋              | 2/3 [27:51<13:55, 835.98s/it]

1
2
3
4
5
6
7
8
9
10
0.7693419968948996
11
12
13
14
15
16
17
18
19
20
0.904848027542196
21
22
23
24
25
26
27
28
29
30
0.7389825075741854
31
32
33
34
35
36
37
38
39
40
0.6270868544027057
41
42
43
44
45
46
47
48
49
50
0.7812340980844343
51
52
53
54
55
56
57
58
59
60
0.6692433814259009
61
62
63
64
65
66
67
68
69
70
0.6608189054374768
71
72
73
74
75
76
77
78
79
80
0.7727570727522339
81
82
83
84
85
86
87
88
89
90
0.7517193723205111
91
92
93
94
95
96
97
98
99
100
0.890120871984127
101
102
103
104
105
106
107
108
109
110
0.6386371959583828
111
112
113
114
115
116
117
118
119
120
0.66690123454279
121
122
123
124
125
126
127
128
129
130
0.7383398692469001
131
132
133
134
135
136
137
138
139
140
0.6660433952601393
141
142
143
144
145
146
147
148
149
150
0.8383237706514088
151
152
153
154
155
156
157
158
159
160
0.7280572495544606
161
162
163
164
165
166
167
168
169
170
0.8604102335882688
171
172
173
174
175
176
177
178
179
180
0.7613457318293321
181
182
183
184
185
186
187
188
189
190
0.55350348



100%|███████████████████████████████████████████| 3/3 [41:26<00:00, 828.71s/it]



### 输入测试文本预处理函数
预处理用于infer的source dataset。文本转成单词id。分batch并pad。末尾用eos补足到最大长度。
这里不需要drop remainder，iterator会自动计算最后一批的样本量。但是后面不能再使用batch size

In [16]:
def BuildTestDataset(source_path, lookup_src, src_eos_id):
    
    # source
    source_dataset = tf.data.TextLineDataset(source_path)
    source_dataset = source_dataset.map(lambda string: tf.string_split([string]).values)
    source_dataset = source_dataset.map(lambda words: (words, tf.size(words)))
    source_dataset = source_dataset.map(lambda words, size: (lookup_src.lookup(words), size))

    # batch and pad
    batched_dataset = source_dataset.padded_batch(
        batch_size,
        padded_shapes=(tf.TensorShape([None]),  # source vectors of unknown size
                        tf.TensorShape([])),     # size(source)
        padding_values=(src_eos_id,  # source vectors padded on the right with src_eos_id
                         0))          # size(source) -- unused
    
    return batched_dataset

### Build the infer model function
Input: batched and padded dataset iterator ((source, source_lengths), (target, target_lengths))

Output: command to run in infer session

In [17]:
def BuildInferModel(test_iterator, tar_sos_id, tar_eos_id):
    (source, source_lengths) = test_iterator.get_next()
    encoder_inputs = tf.transpose(source, [1,0])
    
    embedding_encoder = tf.Variable(embedding_matrix_src, name='embedding_encoder')
    embedding_decoder = tf.Variable(embedding_matrix_tar, name='embedding_decoder')
    
    # Embedding layer
    encoder_emb_inp = tf.nn.embedding_lookup(embedding_encoder, encoder_inputs)
    
    # Encoder
    # Construct forward and backward cells
    forward_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units, name = 'forward_cell')
    backward_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units, name = 'backward_cell')

    bi_outputs, encoder_state = tf.nn.bidirectional_dynamic_rnn(
        forward_cell, backward_cell, encoder_emb_inp, dtype=tf.float64,
        sequence_length=source_lengths, time_major=True)
    encoder_outputs = tf.concat(bi_outputs, -1)
    #encoder_state: LSTM state tuple (cell_state, hidden_state)
    
    # Attention
    attention_states = tf.transpose(encoder_outputs, [1, 0, 2])
    attention_mechanism = tf.contrib.seq2seq.LuongAttention(
        num_units, attention_states, memory_sequence_length=source_lengths, dtype=tf.float64)
    decoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units, name = 'decoder_cell')
    decoder_cell = tf.contrib.seq2seq.AttentionWrapper(decoder_cell, attention_mechanism, attention_layer_size=num_units)
    initial_state = decoder_cell.zero_state(dtype=tf.float64, batch_size=tf.shape(encoder_inputs)[1])
    initial_state = initial_state.clone(cell_state=encoder_state[0])
    
    # Projection layer on the top
    projection_layer = tf.layers.Dense(len(unique_words_tar), use_bias=False, name='projection')
    
    # Decoder to infer
    helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
        embedding_decoder, tf.fill([tf.shape(encoder_inputs)[1]], tf.to_int32(tar_sos_id)), tf.to_int32(tar_eos_id))
    decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, helper, initial_state, output_layer=projection_layer)
    maximum_iterations = tf.round(tf.reduce_max(source_lengths) * 2)
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        decoder, maximum_iterations=maximum_iterations, output_time_major=True, impute_finished=True)
    
    translation_id = tf.to_int64(outputs.sample_id)
    translation = lookup_translate.lookup(translation_id)
    
    return translation

### 设置infer graph

In [20]:
infer_graph = tf.Graph()
with infer_graph.as_default():
    
    # Build the lookup table
    lookup_src, lookup_tar, lookup_translate = BuildLookupTable('en-fr/words_en', 'en-fr/words_fr')
    
    # set the sos and eos
    src_eos_id=lookup_src.lookup(tf.constant('eos')) #0 in source vocab
    tar_sos_id=lookup_tar.lookup(tf.constant('sos')) #0 in target vocab
    tar_eos_id=lookup_tar.lookup(tf.constant('eos')) #1 in target vocab
    
    # Preprocess the text dataset
    batched_dataset = BuildTestDataset(source_path, lookup_src, src_eos_id)
    
    # Build the train model
    test_iterator = batched_dataset.make_initializable_iterator()
    infer_model = BuildInferModel(test_iterator, tar_sos_id, tar_eos_id)
    infer_saver = tf.train.Saver()
    table_initializer = tf.tables_initializer()

### Run the infer session to translate new sentences

In [21]:
infer_sess = tf.Session(graph=infer_graph)
infer_sess.run(table_initializer)
infer_saver.restore(infer_sess, model_path)
infer_sess.run(test_iterator.initializer)
n_batch=0
f = open('en-fr/trans_fr', 'w', encoding='utf-8')
while True:
    try:
        tar_sentences = infer_sess.run(infer_model)
        n_batch+=1
        tar_sentences = np.transpose(tar_sentences)
        for sentence in tar_sentences:
            for word in sentence:
                if word == 'eos': break
                f.write(word.decode('utf-8') + ' ')
            f.write('\n')
        print(n_batch)
    except tf.errors.OutOfRangeError:
        break
f.close()

# Close the session manually and release resources
train_sess.close()
infer_sess.close()

INFO:tensorflow:Restoring parameters from ./tmp-model.ckpt-3
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262

saver = tf.train.Saver()
with tf.Session() as sess: #debug
    sess.run(tf.global_variables_initializer())
    tf.tables_initializer().run()
    sess.run(batched_iterator.initializer)
    n_batch=0
    ei, di, do = sess.run([encoder_inputs, decoder_inputs, decoder_outputs])
            #print (np.shape(ei), np.shape(di), np.shape(do))
    n_batch+=1
    print(n_batch)
    model_path = saver.save(sess, './tmp-model.ckpt')